<figure>
   <IMG SRC="https://mamba-python.nl/images/logo_basis.png" WIDTH=125 ALIGN="right">
</figure>

# Groundwater model

developed by Onno Ebbens

This notebook is created for the Mamba python course to demonstrate how a simple model groundwater model is created by using online metereological data.

table of content:<a class="anchor" id="0"></a>
1. [import files](#1)
2. [create groudnwater model](#2)
3. [plot results model 2018](#3)
4. [create animation](#4)

## 1. import files<a class="anchor" id="1"></a>

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.animation as animation
import matplotlib
import matplotlib.dates as mdates
import os
import pandas as pd
import datetime as dt
import numpy as np
#To run this notebook, you probably should install flopy with 'pip install flopy' in the anaconda prompt
import flopy
import flopy.utils.binaryfile as bf
import groundwater_func as gf
#Also install pastas with "pip install pastas" in the anaconda prompt
import pastas as ps

In [4]:
#settings
%matplotlib inline
plt.style.use('seaborn')

[back to TOC](#0)
## 2. create groundwater model <a class="anchor" id="2"></a>

In [5]:
script_dir    = os.getcwd()
workspace     = script_dir + r'\model'
result_folder = script_dir + r'\results'

In [6]:
modelname     = '2018'

# time discretisation
start_date = dt.datetime(2018,3,1)
end_date   = dt.datetime.now() - dt.timedelta(1)
steady_start_period=True

# creating model grid
nlay = 2
nrow = 1
ncol = 1000

delc = 10
delr = 1

top = 5
botm = [-5, -25]

# specify soil characteristics
hy = 10
tran = [1,250]
vcont = [0.0001]
laycon = [1,3]
sf1=0.15

# specify starting conditions
strt   = 0 
fixed_head_bound_layers = [1]

# recharge_stn=344, start_step=1, steady_start_recharge=0.0007, create_daily_average_rch=False,
# average_rch_start=None, average_rch_end=None

In [7]:
ml = gf.create_mf_model(workspace, modelname,
                        nlay, nrow, ncol, delc, delr,
                        top, botm,
                        start_date, end_date, steady_start_period=steady_start_period,
                        strt=strt, fixed_head_bound_layers=fixed_head_bound_layers,
                        tran=tran, vcont=vcont,
                        hy=hy, laycon=laycon, sf1=sf1,
                        )

#add recharge
ml = gf.add_recharge_knmi(ml, start_date, end_date, recharge_stn=260, start_step=1)

#add river
riv = flopy.modflow.ModflowRiv(ml, 
                               stress_period_data={0: [[0, 0, 0, 3, 100, 2], [0, 0, ncol-1, 3, 100, 2]]})

In [8]:
ml.write_input()
ml.run_model()

Exception: The program mf2005.exe does not exist or is not executable.

In [ ]:
modelname2     = '2017'

start_date2 = start_date.replace(year=2017)
end_date2   = end_date.replace(year=2017)

ml2 = gf.create_mf_model(workspace, modelname2,
                         nlay, nrow, ncol, delc, delr,
                         top, botm,
                         start_date2, end_date2, steady_start_period=steady_start_period,
                         strt=strt, fixed_head_bound_layers=fixed_head_bound_layers,
                         tran=tran, vcont=vcont,
                         hy=hy, laycon=laycon, sf1=sf1,
                         )

#add recharge
ml2 = gf.add_recharge_knmi(ml2, start_date2, end_date2, recharge_stn=260, start_step=1)

#add river
riv = flopy.modflow.ModflowRiv(ml2, 
                               stress_period_data={0: [[0, 0, 0, 3, 100, 2], [0, 0, ncol-1, 3, 100, 2]]})

ml2.write_input()
ml2.run_model()

In [ ]:
modelname3     = 'average_1960-2018'

start_date3 = dt.datetime(1960,1,1)
end_date3   = dt.datetime(2018,1,1)

ml3 = gf.create_mf_model(workspace, modelname3,
                         nlay, nrow, ncol, delc, delr,
                         top, botm,
                         start_date, end_date, steady_start_period=steady_start_period,
                         strt=strt, fixed_head_bound_layers=fixed_head_bound_layers,
                         tran=tran, vcont=vcont,
                         hy=hy, laycon=laycon, sf1=sf1,
                         )

#add recharge
ml3 = gf.add_recharge_knmi(ml3, recharge_stn=260, start_step=1,
                           create_daily_average_rch=True, average_rch_start=start_date3, 
                           average_rch_end=end_date3)

#add river
riv = flopy.modflow.ModflowRiv(ml3, 
                               stress_period_data={0: [[0, 0, 0, 3, 100, 2], [0, 0, ncol-1, 3, 100, 2]]})

ml3.write_input()
ml3.run_model()

In [ ]:
modelname3     = 'average_1960-2018'

start_date3 = dt.datetime(1960,1,1)
end_date3   = dt.datetime(2018,1,1)

ml3 = gf.create_mf_model(workspace, modelname3,
                         nlay, nrow, ncol, delc, delr,
                         top, botm,
                         start_date, end_date,
                         laycon=[1,3], hy=hy, tran=tran, vcont=vcont, 
                         riv_spd={0: [[0, 0, 0, 3, 100, 2], [0, 0, ncol-1, 3, 100, 2]]}, 
                         fixed_head_edge_layers=[1], recharge_stn=260, start_step=1,
                         create_daily_average_rch=True, average_rch_start=start_date3, average_rch_end=end_date3,
                         )
ml3.write_input()
ml3.run_model()

[back to TOC](#0)
## 3. Plot results model 2018<a class="anchor" id="3"></a>

In [ ]:
headobj = bf.HeadFile(os.path.join(workspace, modelname+'.hds'))
times = headobj.get_times()

dis = ml.get_package('DIS')

In [ ]:
lay_count = 1
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)


ax.set_xlabel('')
ax.set_ylabel('groundwater head (m+ref)')
for head_at_layer in headobj.get_data(totim=times[5]):
    ax.plot(dis.sr.xcenter, head_at_layer[0], label = 'model layer %i'%lay_count)
    lay_count+=1
ax.legend()

In [ ]:
lay_count = 1
fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(111)

for time in times:
    ax.plot(dis.sr.xcenter, headobj.get_data(totim=time)[0][0], label = 'timestep %i'%time)
    
ax.legend()
ax.set_xlabel('column')
ax.set_ylabel('groundwater head (m+ref)')

In [ ]:
lay_count = 1
fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(111)

for time in times:
    ax.plot(dis.sr.xcenter, headobj.get_data(totim=time)[1][0], label = 'timestep %i'%time)
    
ax.legend()
ax.set_xlabel('column')
ax.set_ylabel('groundwater head (m+ref)')

[back to TOC](#0)
## 4. Create animation<a class="anchor" id="4"></a>

In [ ]:
os.chdir(result_folder) #necesary because ani.save cannot handle full paths
gf.animate_multiple_gw_rch([ml3,ml2, ml], 'plot_gws_rch.mp4', output_loc=(0,0,400), animation_frequency='D')